In [ ]:
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By

# --- Setup params ---
BASE_URL = "https://www.bis.org"
DOWNLOAD_DIR = "downloads"
INITIAL_DATE = "01/01/2000"
FINAL_DATE = "11/08/2025"
PAGE_LENGTH = 10
MAX_PAGE = 2

os.makedirs(DOWNLOAD_DIR, exist_ok=True)

driver = webdriver.Chrome()

for i in range(1, MAX_PAGE + 1):
    index_url = (
        f"https://www.bis.org/cbspeeches/index.htm?m=256&fromDate={INITIAL_DATE}&tillDate={FINAL_DATE}"
        f"&cbspeeches_page={i}&cbspeeches_page_length={PAGE_LENGTH}"
    )
    print(f"\n=== Processing page {i} ===")
    driver.get(index_url)
    time.sleep(5)  # Wait for JS

    try:
        container = driver.find_element(By.ID, "cbspeeches_list")
        review_links = container.find_elements(By.CSS_SELECTOR, "a.dark[href^='/review/']")
        review_hrefs = [link.get_attribute("href") for link in review_links]
        print(f"Found {len(review_hrefs)} review links on page {i}.")
    except Exception as e:
        print(f"Could not find review links on page {i}: {e}")
        continue

    # --- Iterate over each review link ---
    for review_url in review_hrefs:
        print(f"Visiting: {review_url}")
        driver.get(review_url)
        time.sleep(2)  # Wait for detail page JS (adjust if necessary)

        # Look for pdf link on the detail page
        try:
            pdf_link = driver.find_element(By.CSS_SELECTOR, "a.pdftitle_link[href$='.pdf']")
            pdf_href = pdf_link.get_attribute("href")
            if not pdf_href.startswith("http"):
                pdf_href = BASE_URL + pdf_href
            print("PDF found:", pdf_href)

            # Download the PDF
            response = requests.get(pdf_href)
            filename = os.path.basename(pdf_href)
            save_path = os.path.join(DOWNLOAD_DIR, filename)
            with open(save_path, "wb") as f:
                f.write(response.content)
            print(f"Downloaded PDF to {save_path}")
        except Exception as e:
            print("No PDF found or error:", e)

driver.quit()



=== Processing page 1 ===
Found 10 review links on page 1.
Visiting: https://www.bis.org/review/r250728g.htm
PDF found: https://www.bis.org/review/r250728g.pdf
Downloaded PDF to downloaded_pdfs2/r250728g.pdf
Visiting: https://www.bis.org/review/r250728f.htm
PDF found: https://www.bis.org/review/r250728f.pdf
Downloaded PDF to downloaded_pdfs2/r250728f.pdf
Visiting: https://www.bis.org/review/r250728e.htm
PDF found: https://www.bis.org/review/r250728e.pdf
Downloaded PDF to downloaded_pdfs2/r250728e.pdf
Visiting: https://www.bis.org/review/r250717g.htm
PDF found: https://www.bis.org/review/r250717g.pdf
Downloaded PDF to downloaded_pdfs2/r250717g.pdf
Visiting: https://www.bis.org/review/r250728i.htm
PDF found: https://www.bis.org/review/r250728i.pdf
Downloaded PDF to downloaded_pdfs2/r250728i.pdf
Visiting: https://www.bis.org/review/r250728h.htm
PDF found: https://www.bis.org/review/r250728h.pdf
Downloaded PDF to downloaded_pdfs2/r250728h.pdf
Visiting: https://www.bis.org/review/r250717f.